Der Guide angepasst:

https://towardsdatascience.com/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1


Die Schritte sind:

1. Daten Generieren/Vorbereiten -> Auf die Modellhyperparameter achten -> müssen zu Daten passen
    - Daten Sind sentences(sequenzen) zu bestimmten längen (z.B. 8), und sind in Batches vorliegend
    - Daten Brauchen Start of Stream/ End of Stream tokens oder beide
2. Modell Definieren
    - Positional encoding selber definieren (Vorlage nehmen)
    - Die Transformerstruktur definieren (Hier viele Bauteile von Pytorch verwenden)
    - Das Masking-zeug selber definieren
    - Das Padding zeug evtl selber definieren
3. Training/Validation definieren
    - Modell initialisieren
    - Optimizer Festlegen
    - Kostenfunktion festlegen
    - Trainingsfunktion festlegen (Muss nicht alles in der Trainingsfunktion direkt passieren, aber es muss passieren)
        - Es muss beim Training diese Verschiebung der Tokens passieren, dass der nächste output für eine Sequenz ausgegeben wird
        - Target-tensor wird während der Prediction ans Modell gegeben
        - Target-Maske muss Generiert werden 
        - Padding maske muss evtl auch generiert werden
    - Validationfunktion festlegen
        - Ist das gleiche wie im Training, nur werden hier keine Gradienten geupdated oder gelesen
4. Training/Validation ausführen
5. Inferenz ausführen

# 0. Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import math
import numpy as np

import random

gpu nutzen

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Daten Generieren/Vorbereiten

In [3]:
def generate_random_data(n):
    SOS_token = np.array([2])
    EOS_token = np.array([3])
    length = 8

    data = []

    # 1,1,1,1,1,1 -> 1,1,1,1,1
    for i in range(n // 3):
        X = np.concatenate((SOS_token, np.ones(length), EOS_token))
        y = np.concatenate((SOS_token, np.ones(length), EOS_token))
        data.append([X, y])

    # 0,0,0,0 -> 0,0,0,0
    for i in range(n // 3):
        X = np.concatenate((SOS_token, np.zeros(length), EOS_token))
        y = np.concatenate((SOS_token, np.zeros(length), EOS_token))
        data.append([X, y])

    # 1,0,1,0 -> 1,0,1,0,1
    for i in range(n // 3):
        X = np.zeros(length)
        start = random.randint(0, 1)

        X[start::2] = 1

        y = np.zeros(length)
        if X[-1] == 0:
            y[::2] = 1
        else:
            y[1::2] = 1

        X = np.concatenate((SOS_token, X, EOS_token))
        y = np.concatenate((SOS_token, y, EOS_token))

        data.append([X, y])

    np.random.shuffle(data)

    return data


def batchify_data(data, batch_size=16, padding=False, padding_token=-1):
    batches = []
    for idx in range(0, len(data), batch_size):
        # We make sure we dont get the last bit if its not batch_size size
        if idx + batch_size < len(data):
            # Here you would need to get the max length of the batch,
            # and normalize the length with the PAD token.
            if padding:
                max_batch_length = 0

                # Get longest sentence in batch
                for seq in data[idx : idx + batch_size]:
                    if len(seq) > max_batch_length:
                        max_batch_length = len(seq)

                # Append X padding tokens until it reaches the max length
                for seq_idx in range(batch_size):
                    remaining_length = max_batch_length - len(data[idx + seq_idx])
                    data[idx + seq_idx] += [padding_token] * remaining_length

            batches.append(np.array(data[idx : idx + batch_size]).astype(np.int64))

    print(f"{len(batches)} batches of size {batch_size}")

    return batches


train_data = generate_random_data(9000)
val_data = generate_random_data(3000)

train_dataloader = batchify_data(train_data)
val_dataloader = batchify_data(val_data)

# 2. Modell Definieren

## 2.1 Positional Encoding

In [4]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Info
        self.dropout = nn.Dropout(p=dropout)
        
        # Encoding - From formula -> This is basically applying the formula for Positional encoding (The one with Sinus and Cosinus)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # Baically a positions list 0, 1, 2, 3, 4, 5, ...
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # 1000^(2i/dim_model)
        
        # # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        
        #  # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pe[:, 1::2] = torch.cos(position * div_term)
        
         # Saving buffer (same as parameter without gradients needed)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """
        
        # Residual connection + pos encoding
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## 2.2 Transformer Definieren

In [5]:
class Transformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Transformer, self).__init__()
        
        # Hier werden glaube ich die Layer definiert. Ist Im guide glaube ich in anderer Reihenfolge -> hab sie jetzt in die gleiche Reihenfolge wie im guide gepackt
        
        ## Layers des Gesamten Modells
        
        # Positional Encoding zur Hinzufügung von Positionsinformationen zu den Token-Einbettungen
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)
        
        # Token-Einbettung für Quell- und Zielvokabular
        # I use a nn.Embedding instead of the self defined TokenEmbedding
        self.src_tok_emb = nn.Embedding(src_vocab_size, emb_size) 
        self.tgt_tok_emb = nn.Embedding(tgt_vocab_size, emb_size)
        
        # Initialisierung des nn.Transformer Moduls mit den gegebenen Hyperparametern
        self.transformer = nn.Transformer(d_model=emb_size,
                                          nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout,
                                          batch_first=True)
        
        # Linearer Layer zur Projektion der Ausgabedimensionen auf die Zielvokabulargröße
        # Generator ist also glaube ich die Outputlayer, die die Ausgabe in die Wahrscheinlichkeiten für die einzelnen Tokens übersetzt
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        
        

    def forward(self,
                src: Tensor,
                trg: Tensor,
                # src_mask: Tensor,
                tgt_mask=None,
                src_padding_mask=None,
                tgt_padding_mask=None):
        
        
        # Einbettung und Positional Encoding für die Quellsequenz
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        
        # Einbettung und Positional Encoding für die Zielsequenz
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        
        # Hier bin ich noch etwas verwirrt, warum die dimensionen Permutiert werden müssen
        # Aus der Erklärung für die batch_first variable von nn.Transformer:
        # If True, then the input and output tensors are provided as (batch, seq, feature). Default: False (seq, batch, feature).
        
        # (deprecated) src_emb = src_emb.permute(1,0,2)
        # (deprecated) tgt_emb = tgt_emb.permute(1,0,2)
        #print("src_emb shape:", src_emb.shape)
        #print("tgt_emb shape:", tgt_emb.shape)
        
        # Durchführen der Transformationsoperation
        # src_emb und tgt_emb sind die eingebetteten Sequenzen mit Positionsinformationen
        # src_mask und tgt_mask sind die Masken, die verhindern, dass zukünftige Tokens betrachtet werden
        # src_padding_mask, tgt_padding_mask und memory_key_padding_mask sind die Masken für Padding-Tokens
        outs = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask, src_key_padding_mask=src_padding_mask, tgt_key_padding_mask=tgt_padding_mask)
        
        # Projektion der Ausgabe auf die Zielvokabulargröße
        return self.generator(outs)

2.3 Masken definieren

In [6]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    
    # EX for size=5:
    # [[0., -inf, -inf, -inf, -inf],
    #  [0.,   0., -inf, -inf, -inf],
    #  [0.,   0.,   0., -inf, -inf],
    #  [0.,   0.,   0.,   0., -inf],
    #  [0.,   0.,   0.,   0.,   0.]]
    
    return mask

# Die Funktion create_mask erstellt sowohl Quell- als auch Ziel-Pad-Masken, indem sie prüft, ob Elemente in der Quell- und Zielsequenz gleich dem Pad-Token sind. Diese Masken werden transponiert, um die richtige Dimension zu erhalten.
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

.type(torch.bool)# 3. Training/Validation definieren

## 3.1 Hyperparameter und special tokens festlegen und Modell laden

In [7]:
SRC_VOCAB_SIZE = 4 # Ist glaube ich das num_tokens aus dem Guide (Also wie viele Verschiedene Tokens es insgesamt gibt
TGT_VOCAB_SIZE = 4 # Auch 4, da die eingabe und zielsequenz die Gleichen möglichkeiten für Tokens haben
EMB_SIZE = 8 #die Dimesnion des Modells Die anzahl der Erwarteten features der inputs/outputs also quasi die anzahl der Wörter in einer sequenz glaube ich -> also 8 bei uns (hier werden die spezial-tokens nicht gezählt ?)
NHEAD = 2 # Anzahl der heads in einem Attention block
FFN_HID_DIM = 512 # Anzahl der hidden layers des Feed-forward networks 
BATCH_SIZE = 16 # wird nicht ans Modell weitergegeben. evtl für uns nicht wichtig, weil wir die Daten schon gebatcht haben?
NUM_ENCODER_LAYERS = 3 # wie viele Encoder blöcke
NUM_DECODER_LAYERS = 3 # wie viele Decoder Blöcke

Die ganzen Tokens müssen auf die Daten abgestimmt werden

In [8]:
UNK_IDX = 5 #Brauchen wir glaube ich nicht
PAD_IDX = 4 #Brauchen wir auch nicht, weil die sequenzen alle genau richtig lang sind
BOS_IDX = 2 # So gesetzt wie im Guide beispiel mit den sequenzen
EOS_IDX = 3 #auch gesetz wie im guide (hoffe ich)

In [9]:
# Modell initialisiern
transformer = Transformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
# auf GPU laden
transformer = transformer.to(DEVICE)
# Kostenfunktion als CrossEntropyLoss
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
# Optimizer als Adam optimizer festlegen
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

## 3.2 Trainingsloop defineren

In [10]:
def train_loop(model, opt, loss_fn, dataloader):
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        X, y = batch[:, 0], batch[:, 1]
        X, y = torch.tensor(X).to(DEVICE), torch.tensor(y).to(DEVICE)

        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        y_input = y[:,:-1]
        y_expected = y[:,1:]
        
        #print("Training: X shape:", X.shape)
        #print("Training: y_input shape:", y_input.shape)
        #print("Training: y_expected shape:", y_expected.shape)
        
        # Get mask to mask out the next words
        sequence_length = y_input.size(1)
        tgt_mask = generate_square_subsequent_mask(sequence_length).to(DEVICE)

        # Standard training except we pass in y_input and tgt_mask
        logits = model(X, y_input, tgt_mask)
        
        #print("Training: prediction (model output) shape:", logits.shape)
        
        #(deprecated) Permute pred to have batch size first again
        #(deprecated) pred = pred.permute(1, 2, 0)
        # logits ist die Ausgabe des modells, y_expected ist die erwartete ausgabe
        # Die dimensionen müssen verändert werden, da die loss funktion die Tensoren in anderer Form erwartet
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), y_expected.reshape(-1))

        opt.zero_grad()
        loss.backward()
        opt.step()
    
        total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

## 3.3 Validation loop definieren

In [11]:
def validation_loop(model, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch[:, 0], batch[:, 1]
            X, y = torch.tensor(X, dtype=torch.long, device=DEVICE), torch.tensor(y, dtype=torch.long, device=DEVICE)

            # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
            y_input = y[:,:-1]
            y_expected = y[:,1:]
            
            # Get mask to mask out the next words
            sequence_length = y_input.size(1)
            tgt_mask = generate_square_subsequent_mask(sequence_length).to(DEVICE)

            # Standard training except we pass in y_input and src_mask
            logits = model(X, y_input, tgt_mask)

            # Permute pred to have batch size first again
            #pred = pred.permute(1, 2, 0)
            # logits ist die Ausgabe des modells, y_expected ist die erwartete ausgabe
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), y_expected.reshape(-1))
            total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

# 4. Modell Trainieren

In [12]:
from timeit import default_timer as timer
NUM_EPOCHS = 10

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_loop(transformer, optimizer, loss_fn, train_dataloader)
    end_time = timer()
    val_loss = validation_loop(transformer, loss_fn, val_dataloader)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

### Training: X shape: torch.Size([16, 10])
- **X:** Dies ist die Eingabesequenz (Quelle) des Modells.
  - `16`: Batchgröße (Anzahl der Sequenzen in einem Batch).
  - `10`: Sequenzlänge (Anzahl der Token in jeder Sequenz).

### Training: y_input shape: torch.Size([16, 9])
- **y_input:** Dies ist die Zielsequenz, die als Eingabe für den Decoder verwendet wird, aber ohne das letzte Token (da das letzte Token nur zum Vergleich verwendet wird).
  - `16`: Batchgröße.
  - `9`: Sequenzlänge der Zielsequenz (eine weniger als die Quellsequenz, da das letzte Token entfernt wurde).

### Training: y_expected shape: torch.Size([16, 9])
- **y_expected:** Dies ist die Zielsequenz, gegen die das Modell bewertet wird. Es ist die Zielsequenz ohne das erste Token (typischerweise das Starttoken) und wird zum Berechnen des Verlusts verwendet.
  - `16`: Batchgröße.
  - `9`: Sequenzlänge der Zielsequenz (wie bei `y_input`).

### src_emb shape: torch.Size([16, 10, 8])
- **src_emb:** Dies ist die eingebettete Quellsequenz nach Anwendung des Positional Encodings.
  - `16`: Batchgröße.
  - `10`: Sequenzlänge der Quellsequenz.
  - `8`: Einbettungsgröße (Anzahl der Merkmale pro Token nach der Einbettung).

### tgt_emb shape: torch.Size([16, 9, 8])
- **tgt_emb:** Dies ist die eingebettete Zielsequenz nach Anwendung des Positional Encodings.
  - `16`: Batchgröße.
  - `9`: Sequenzlänge der Zielsequenz.
  - `8`: Einbettungsgröße.

### Training: prediction logits (model output) shape: torch.Size([16, 9, 4])
- **prediction:** Dies ist die Ausgabe des Modells nach der Transformation und Projektion auf die Zielvokabulargröße.
  - `16`: Batchgröße.
  - `9`: Sequenzlänge der Zielsequenz.
  - `4`: Größe des Zielvokabulars (Anzahl der möglichen Token im Zielvokabular).

### Erklärung der Tensorformen

- **Batchgröße (16):** Anzahl der Sequenzen, die gleichzeitig verarbeitet werden.
- **Sequenzlänge (10 für Quelle, 9 für Ziel):** Anzahl der Token in jeder Sequenz. Die Zielsequenz ist um eins kürzer, da das letzte Token entfernt wird, um als y_expected verwendet zu werden.
- **Einbettungsgröße (8):** Dimension der eingebetteten Vektoren nach der Positional Encoding.
- **Zielvokabulargröße (4):** Anzahl der möglichen Token im Zielvokabular. Dies ist die Anzahl der Klassen, die das Modell vorherzusagen versucht.

### Zusammenfassung

Die Batchgröße bleibt über alle Tensoren hinweg konsistent. Die Sequenzlänge der Eingabesequenzen (Quellsequenzen) ist etwas länger als die der Zielsequenzen, da die Zielsequenzen angepasst werden, um `y_input` und `y_expected` zu erzeugen. Die Einbettungsgröße bleibt gleich für Quell- und Zielsequenzen, während die Modellvorhersage die Form `(batch_size, seq_len, vocab_size)` hat, wobei `vocab_size` die Anzahl der Klassen im Zielvokabular ist.

### Erklärung Kostenfunktion

1. **Vorhersagen des Modells (logits):**
   - `logits = model(X, y_input, tgt_mask)`
   - Dies ist die Ausgabe des Modells mit der Form `(batch_size, seq_len, vocab_size)`.

2. **Erwartete Zielwerte (y_expected):**
   - `y_expected` ist die erwartete Ausgabe mit der Form `(batch_size, seq_len)`.

3. **Anpassen der Dimensionen:**
   - Um die `CrossEntropyLoss`-Funktion zu verwenden, müssen die Vorhersagen (`logits`) und die Zielwerte (`y_expected`) in bestimmten Formen vorliegen.

### Berechnung der Kostenfunktion

#### 1. Ausgabe des Modells reshapen:

- `logits.reshape(-1, logits.shape[-1])`
  - `logits` hat die Form `(batch_size, seq_len, vocab_size)`.
  - `logits.reshape(-1, logits.shape[-1])` ändert die Form zu `(batch_size * seq_len, vocab_size)`.
  - Dies ist notwendig, da die `CrossEntropyLoss`-Funktion die Form `(N, C)` erwartet, wobei `N` die Anzahl der Datenpunkte und `C` die Anzahl der Klassen ist.

#### 2. Zielwerte reshapen:

- `y_expected.reshape(-1)`
  - `y_expected` hat die Form `(batch_size, seq_len)`.
  - `y_expected.reshape(-1)` ändert die Form zu `(batch_size * seq_len)`.
  - Dies ist notwendig, da die `CrossEntropyLoss`-Funktion die Form `(N)` erwartet, wobei `N` die Anzahl der Datenpunkte ist.

# 5. Inferenz

In [15]:
def predict(model, input_sequence, max_length=15, SOS_token=2, EOS_token=3):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    model.eval()
    
    y_input = torch.tensor([[SOS_token]], dtype=torch.long, device=DEVICE)
    
    print("input sequence:", input_sequence.shape)
    print("y_input shape:", y_input.shape)
    
    num_tokens = len(input_sequence[0])

    for _ in range(max_length):
        # Get source mask
        tgt_mask = generate_square_subsequent_mask(y_input.size(1)).to(DEVICE)
        
        pred = model(input_sequence, y_input, tgt_mask)
        
        next_item = pred.topk(1)[1].view(-1)[-1].item() # num with highest probability
        next_item = torch.tensor([[next_item]], device=DEVICE)

        # Concatenate previous input with predicted best word
        y_input = torch.cat((y_input, next_item), dim=1)

        # Stop if model predicts end of sentence
        if next_item.view(-1).item() == EOS_token:
            break

    return y_input.view(-1).tolist()
  
  
# Here we test some examples to observe how the model predicts
examples = [
    torch.tensor([[2, 1, 0, 0, 0, 1, 0, 0, 0, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 1, 1, 1, 1, 1, 1, 1, 1, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 1, 0, 1, 0, 1, 0, 1, 0, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 0, 1, 0, 1, 0, 1, 0, 1, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 0, 1, 3]], dtype=torch.long, device=DEVICE)
]

for idx, example in enumerate(examples):
    result = predict(transformer, example)
    print(f"Example {idx}")
    print(f"Input: {example.view(-1).tolist()[1:-1]}")
    print(f"Continuation: {result[1:-1]}")
    print()

# Chat gpt erwartung wenn mit unseren Daten gearbeitet werden soll

Wenn Ihre Eingangsdaten nun die Form `(16, 2, 10, 88)` haben, bedeutet dies, dass Sie nicht mehr nur Sequenzen von Nullen und Einsen (binares Format) verwenden, sondern Sequenzen von Arrays mit Länge 88. Diese Art von Datenstruktur könnte darauf hinweisen, dass Sie mit eingebetteten Sequenzen arbeiten, bei denen jedes Token in der Sequenz durch einen Vektor der Länge 88 dargestellt wird.

### Anpassungen am Modell

In diesem Fall müssen Sie sicherstellen, dass das Modell in der Lage ist, mit den eingebetteten Sequenzen der Länge 88 korrekt umzugehen. Hier sind die notwendigen Anpassungen:

1. **Anpassung der Eingangsdatenform**: 
    - Die Eingangsdaten haben nun die Form `(batch_size, 2, seq_len, feature_dim)`, also `(16, 2, 10, 88)`.

2. **Extrahieren der Quell- und Zielsequenzen**:
    - Sie müssen sicherstellen, dass Sie die Quell- und Zielsequenzen korrekt extrahieren.

3. **Anpassung der Einbettungsschichten**:
    - Da die Eingabedaten bereits eingebettet sind, benötigen Sie möglicherweise keine zusätzlichen Einbettungsschichten (`nn.Embedding`) mehr.

4. **Anpassung der linearen Schicht**:
    - Die lineare Schicht muss entsprechend der Größe der eingebetteten Dimensionen angepasst werden.
